# MCP in LangChain 1.0

## What is MCP?
* Model Context Protocol (MCP) is an open protocol that standardizes how we can connect external tools to our LLM applications and Agents (as you know, Agents are just a type of LLM application).

## How can Agents use MCP in LangChain 1.0?
* LangChain agents can use tools defined on MCP servers using the [langchain-mcp-adapters](https://github.com/langchain-ai/langchain-mcp-adapters) library.
* You can find available MCP tools in many sources like [this website](https://mcp.so/server/time/modelcontextprotocol).

#### How to use external tools with MCP: the basics
* First, we need to install the langchain-mcp-adapters package:
`pip install langchain-mcp-adapters`

* Then, we will use `MultiServerMCPClient`, a module that for each tool invocation creates a fresh MCP session, executes the tool, and then cleans up:

```python
from langchain_mcp_adapters.client import MultiServerMCPClient  
from langchain.agents import create_agent


client = MultiServerMCPClient(  
    {
        "math": {
            "transport": "stdio",  # Local subprocess communication
            "command": "python",
            # Absolute path to your math_server.py file
            "args": ["/path/to/math_server.py"],
        },
        "weather": {
            "transport": "http",  # HTTP-based remote server
            # Ensure you start your weather server on port 8000
            "url": "http://localhost:8000/mcp",
        }
    }
)

tools = await client.get_tools()  

agent = create_agent(
    "gpt-4o-mini",
    tools  
)

math_response = await agent.ainvoke(
    {"messages": [{
        "role": "user", 
        "content": "what's (3 + 5) x 12?"}]}
)

weather_response = await agent.ainvoke(
    {"messages": [{
        "role": "user", 
        "content": "what is the weather in nyc?"}]}
)
```

## Let's see this in a basic example

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from pprint import pprint
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            # the next lines are pasted from the website of the MCP provider
            # see https://mcp.so/server/time/modelcontextprotocol
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

agent = create_agent(
    model="gpt-4o-mini",
    tools=tools,
)

question = HumanMessage(content="What time is it in Madrid?")

response = await agent.ainvoke(
    {"messages": [question]}
)

print(response['messages'][-1].content)

The current time in Madrid is 06:23 AM on Sunday, January 4, 2026.


## OK. Let's explain the previous code in simple terms

#### Imports (lines 1-3)

```python
from langchain.agents import create_agent
```
**What it does:** Imports a function that allows you to create an AI agent.
**In simple terms:** An agent is like an intelligent assistant that can use tools to solve problems. This function is the "factory" that builds that assistant.

```python
from langchain.messages import HumanMessage
```
**What it does:** Imports a class to create user messages.
**In simple terms:** When you want to "talk" to the agent, you need to package your question in a special format. `HumanMessage` is that packaging.

```python
from pprint import pprint
```
**What it does:** Imports a function to print data in a nice way.
**In simple terms:** Although it's not used in this code, `pprint` helps to see complex results in a more readable way. (Note: the code uses regular `print` at the end, not `pprint`).


#### Connection to MCP server (lines 4-18)

```python
from langchain_mcp_adapters.client import MultiServerMCPClient
```
**What it does:** Imports a client to connect to MCP servers.
**In simple terms:** MCP (Model Context Protocol) is a standard that allows your agent to use external tools. This client is the "bridge" that connects your agent with those tools.

```python
client = MultiServerMCPClient(
```
**What it does:** Creates an instance of the MCP client.
**In simple terms:** You're creating your connection to the tool servers. "Multi" means you can connect to multiple servers at once.

```python
    {
        "time": {
```
**What it does:** Defines a server called "time".
**In simple terms:** You're telling the client: "Connect to a server called 'time' that has tools related to dates and times".

```python
            "transport": "stdio",
```
**What it does:** Defines how it communicates with the server.
**In simple terms:** "stdio" means "Standard Input/Output". It's like saying "I'm going to talk to this server using simple text messages through the terminal".

```python
            "command": "uvx",
```
**What it does:** Specifies the command to execute the server.
**In simple terms:** `uvx` is a Python tool that runs applications. It's like saying "use uvx to start the server".

```python
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
```
**What it does:** Provides the arguments (parameters) for the command.
**In simple terms:** It tells `uvx` to execute `mcp-server-time` (the time server) configured for the New York timezone. It's like saying: "run this program with this specific configuration".


#### Getting tools (line 19)

```python
tools = await client.get_tools()
```
**What it does:** Gets all available tools from the MCP server.
**In simple terms:** The client asks the server: "What tools do you have available?" and saves them in the `tools` variable. The `await` means that the program waits to receive the response before continuing (because it's an asynchronous operation).


#### Agent creation (lines 20-23)

```python
agent = create_agent(
    model="gpt-4o-mini",
    tools=tools,
)
```
**What it does:** Creates the agent with an AI model and the MCP tools.
**In simple terms:** Here you build your intelligent assistant. You tell it:
- "Use the GPT-4o-mini brain" (OpenAI's AI model)
- "These are the tools you can use" (the ones you got from the MCP server)

The agent now knows how to think (GPT-4o-mini) and what tools it has available.


#### Question and execution (lines 24-28)

```python
question = HumanMessage(content="What time is it in Madrid?")
```
**What it does:** Creates a message with the user's question.
**In simple terms:** You package your question "What time is it in Madrid?" in the format that the agent understands.

```python
response = await agent.ainvoke(
    {"messages": [question]}
)
```
**What it does:** Sends the question to the agent and waits for its response.
**In simple terms:** You tell the agent: "Here's my question, solve it". The agent:
1. Reads the question
2. Decides if it needs to use any tool
3. Uses the MCP server to get the time
4. Returns the answer to you

The `await` means you wait for this entire process to finish.


#### Display result (line 29)

```python
print(response['messages'][-1].content)
```
**What it does:** Prints the last message of the conversation.
**In simple terms:** 
- `response['messages']` is a list with all the messages of the conversation
- `[-1]` takes the last message (the agent's final response)
- `.content` extracts the text from that message
- `print()` displays it on screen


#### General summary

This code does the following in order:

1. **Sets up the connection** to an MCP server that has time/date tools
2. **Gets the available tools** from that server
3. **Creates an intelligent agent** that knows how to use those tools
4. **Asks a question**: "What time is it in Madrid?"
5. **The agent reasons**: "I need to use the time tool to answer this"
6. **Shows the answer** that the agent obtained

It's a perfect example of how LangChain agents can use external tools (MCP) to answer questions that require real-time information.

#### If you use pprint to see the detailed response, you will see that we are indeed using the external tool via MCP

In [11]:
pprint(response)

{'messages': [HumanMessage(content='What time is it in Madrid?', additional_kwargs={}, response_metadata={}, id='441c9705-4670-462d-84d3-4d2fb9fdf593'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 217, 'total_tokens': 234, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bbc38b4db', 'id': 'chatcmpl-CuAs19asAid3hUvFpBqicOZx2a8u6', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b8775-96b3-77a2-839d-2f5ae0c89689-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Europe/Madrid'}, 'id': 'call_xbOgqvy1doAZxhw4izo4jUWx', 'type': 'tool_call'}], usage_metadata={'input_tokens': 217, '

## How to run this code from Visual Studio Code
* Open Terminal.
* Make sure you are in the project folder.
* Make sure you have the poetry env activated.
* Enter and run the following command:
    * `python 008-mcp.py`
 
#### Important note about a small change in the .py file

Jupyter notebooks handle async code differently than regular Python scripts.

**In Jupyter notebooks:**
- Jupyter (specifically IPython) has a built-in event loop that's always running
- This allows you to use `await` directly at the top level of cells without wrapping it in an `async def` function
- Jupyter automatically handles the async execution for you

**In regular Python scripts (terminal):**
- There's no event loop running by default
- You must explicitly create an event loop using `asyncio.run()`
- All `await` statements must be inside `async def` functions

**Example:**

```python
# ✅ Works in Jupyter notebook
tools = await client.get_tools()

# ✅ Works in terminal/script
async def main():
    tools = await client.get_tools()
    
asyncio.run(main())
```

This is one of the convenient features of Jupyter for working with async code - it makes experimentation easier. But when you move that code to a `.py` file to run from the terminal, you need to add the async function wrapper and `asyncio.run()`.

This is a common "gotcha" when transitioning code from Jupyter notebooks to Python scripts!